# Лабораторная работа 4 (RandomForest)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from utils import regression_cross_validate, display_metrics_table, classification_cross_validate, display_metrics_classification_table


In [2]:
import warnings
warnings.filterwarnings("ignore")

### Regression

#### 1. Обработка данных

In [3]:
df = pd.read_csv('data/Salary_Data.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32,Male,Bachelor's,Software Engineer,5.0,90000
1,28,Female,Master's,Data Analyst,3.0,65000
2,45,Male,PhD,Senior Manager,15.0,150000
3,36,Female,Bachelor's,Sales Associate,7.0,60000
4,52,Male,Master's,Director,20.0,200000


In [4]:
X, y = df.drop(columns=['Salary', 'Job Title']), df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

gender_le = LabelEncoder()
el_le = LabelEncoder()

X_train['Gender'] = gender_le.fit_transform(X_train['Gender'])
X_test['Gender'] = gender_le.transform(X_test['Gender'])

X_train['Education Level'] = el_le.fit_transform(X_train['Education Level'])
X_test['Education Level'] = el_le.transform(X_test['Education Level'])


#### 2. Построение бейзлайна 

Для оценки модели будем использовать метод кросс валидации, который позволяет более качественно оценить полученные метрики.

In [5]:
metrics = regression_cross_validate(RandomForestRegressor, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=3)
display_metrics_table(*metrics)

rf = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=3)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |        Std Dev |
|:---------|----------------:|---------------:|
| MAE      | 11495.4         | 1124.84        |
| MSE      |     2.69259e+08 |    5.88017e+07 |
| R2       |     0.883911    |    0.0281219   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 309918648.63
Средняя абсолютная ошибка (MAE): 12029.18
Коэффициент детерминации (R^2): 0.86


Как можем увидеть значение метрики $R^2$ около 0.86, что означает что около 86% дисперсии данных объясняется моделью.

#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить глубину и число деревьев

In [6]:
onehot = OneHotEncoder(sparse_output=False, drop='first')

categorical_features = ['Gender', 'Education Level']

encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)

scaler = StandardScaler()
num_features = ['Age', 'Years of Experience']
X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])

In [7]:
metrics = regression_cross_validate(RandomForestRegressor, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, random_state=42, n_estimators=300, max_depth=7)
display_metrics_table(*metrics)

rf = RandomForestRegressor(n_estimators=300, max_depth=7, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |        Std Dev |
|:---------|----------------:|---------------:|
| MAE      | 10742           | 1072.42        |
| MSE      |     2.60804e+08 |    5.24434e+07 |
| R2       |     0.886885    |    0.0282617   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 266888563.88
Средняя абсолютная ошибка (MAE): 10644.36
Коэффициент детерминации (R^2): 0.88


Можно увидеть, что в целом значения метрик улучшились. Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [8]:
class MyRandomForestRegressor:
    def __init__(self, n_estimators=100, max_depth=None):
        self.n_estimators = n_estimators  # Количество деревьев
        self.max_depth = max_depth  # Глубина дерева
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), size=len(X), replace=True)
            X_bootstrap = X[bootstrap_indices]
            y_bootstrap = y[bootstrap_indices]
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.zeros((len(X), self.n_estimators))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)
        return predictions.mean(axis=1)

In [9]:
metrics = regression_cross_validate(MyRandomForestRegressor, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_estimators=100, max_depth=3)
display_metrics_table(*metrics)

rf = MyRandomForestRegressor(n_estimators=100, max_depth=3)
rf.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = rf.predict(X_test.to_numpy())

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |           Mean |        Std Dev |
|:---------|---------------:|---------------:|
| MAE      | 11395.2        | 1354.32        |
| MSE      |     2.6682e+08 |    6.36048e+07 |
| R2       |     0.885078   |    0.0297093   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 308030501.12
Средняя абсолютная ошибка (MAE): 12143.92
Коэффициент детерминации (R^2): 0.86


In [10]:
metrics = regression_cross_validate(MyRandomForestRegressor, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_estimators=300, max_depth=7)
display_metrics_table(*metrics)

rf = MyRandomForestRegressor(n_estimators=300, max_depth=7)
rf.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = rf.predict(X_test.to_numpy())

# Метрики
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Вывод метрик
print("\n=== Результаты на Тесте ===")
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Средняя абсолютная ошибка (MAE): {mae:.2f}")
print(f"Коэффициент детерминации (R^2): {r2:.2f}")

| Metric   |            Mean |        Std Dev |
|:---------|----------------:|---------------:|
| MAE      | 10759.2         | 1109.63        |
| MSE      |     2.58831e+08 |    5.68971e+07 |
| R2       |     0.887654    |    0.0301864   |

=== Результаты на Тесте ===
Среднеквадратичная ошибка (MSE): 269138945.61
Средняя абсолютная ошибка (MAE): 10625.08
Коэффициент детерминации (R^2): 0.88


Результаты показывают что собственная имплементация модели в среднем работает на том же уровне качества, что и модель из `sklearn`

### Classification

#### 1. Обработка данных

In [11]:
df = pd.read_csv('data/Student_Depression_Dataset.csv')
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [12]:
X, y = df.drop(columns=['Depression', 'id']), df['Depression']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

categorical_features = ['Gender', 'City', 'Profession', 'Sleep Duration', 'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']
num_features = ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction', 'Work/Study Hours', 'Financial Stress']

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=99)

X_train[categorical_features] = le.fit_transform(X_train[categorical_features])
X_test[categorical_features] = le.transform(X_test[categorical_features])

imputer = SimpleImputer(strategy='most_frequent') 
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)


#### 2. Построение бейзлайна 

In [13]:
metrics = classification_cross_validate(RandomForestClassifier, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, random_state=42, n_estimators=100, max_depth=3)
display_metrics_classification_table(*metrics)

rf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=3)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.822748 | 0.00599261 |
| Precision | 0.823032 | 0.00627539 |
| Recall    | 0.822748 | 0.00599261 |
| F1-score  | 0.820647 | 0.00615235 |

=== Результаты на Тесте ===
1. Accuracy: 81.38%
2. Precision: 81.41%
3. Recall: 81.38%
4. F1-score: 81.16%


#### 3. Формулировка гипотез

Сформулируем несколько гипотез, которые могут помочь улучшить качество модели

1) Поменять Encoder категориальных признаков с `LabelEncoder` на `OneHotEncoder`
2) Отмасштабировать численные признаки
3) Увеличить глубину и число деревьев

In [14]:
onehot = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')


encoded_train_data = onehot.fit_transform(X_train[categorical_features])
encoded_test_data = onehot.transform(X_test[categorical_features])

encoded_df = pd.DataFrame(encoded_train_data, columns=onehot.get_feature_names_out(categorical_features))
X_train_upd = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_upd = pd.concat([X_train_upd, encoded_df], axis=1)

encoded_df = pd.DataFrame(encoded_test_data, columns=onehot.get_feature_names_out(categorical_features))
X_test_upd = X_test.drop(columns=categorical_features).reset_index(drop=True)
X_test_upd = pd.concat([X_test_upd, encoded_df], axis=1)

X_train_upd[num_features] = scaler.fit_transform(X_train[num_features])
X_test_upd[num_features] = scaler.transform(X_test[num_features])


In [15]:
metrics = classification_cross_validate(RandomForestClassifier, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, random_state=42, n_estimators=300, max_depth=10)
display_metrics_classification_table(*metrics)

rf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42)
rf.fit(X_train_upd, y_train)

y_pred = rf.predict(X_test_upd)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")


| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.841195 | 0.00385471 |
| Precision | 0.841791 | 0.00401028 |
| Recall    | 0.841195 | 0.00385471 |
| F1-score  | 0.839419 | 0.00392379 |

=== Результаты на Тесте ===
1. Accuracy: 82.87%
2. Precision: 82.88%
3. Recall: 82.87%
4. F1-score: 82.71%


Можно увидеть что в среднем значения метрик улучшились. 
Результаты на тестовой выборке также показывают приросты.

#### 4. Реализация своего класса

In [16]:
class MyRandomForestClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=None, min_samples_split=2):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        self.feature_indices = []

        for _ in range(self.n_estimators):
            # Создаем бутстреп-выборку
            X_sample, y_sample = self.bootstrap_sample(X, y)

            # Создаем и обучаем дерево
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        # Возвращаем большинство голосов
        predictions = np.apply_along_axis(lambda x: np.bincount(x, minlength=len(np.unique(x))).argmax(), axis=0, arr=tree_preds)
        return predictions

In [17]:
metrics = classification_cross_validate(MyRandomForestClassifier, X_train.to_numpy(), y_train.to_numpy(), n_folds=5, n_estimators=100, max_depth=3)
display_metrics_classification_table(*metrics)

rf = MyRandomForestClassifier(n_estimators=100, max_depth=3)
rf.fit(X_train.to_numpy(), y_train.to_numpy())

y_pred = rf.predict(X_test.to_numpy())

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")

| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.824659 | 0.00266254 |
| Precision | 0.825127 | 0.00275535 |
| Recall    | 0.824659 | 0.00266254 |
| F1-score  | 0.822459 | 0.00288935 |

=== Результаты на Тесте ===
1. Accuracy: 81.26%
2. Precision: 81.28%
3. Recall: 81.26%
4. F1-score: 81.04%


In [18]:
metrics = classification_cross_validate(MyRandomForestClassifier, X_train_upd.to_numpy(), y_train.to_numpy(), n_folds=5, n_estimators=300, max_depth=10)
display_metrics_classification_table(*metrics)

rf = MyRandomForestClassifier(n_estimators=300, max_depth=10)
rf.fit(X_train_upd.to_numpy(), y_train.to_numpy())

y_pred = rf.predict(X_test_upd.to_numpy())

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Выводим результаты
print("\n=== Результаты на Тесте ===")
print(f"1. Accuracy: {accuracy:.2%}")
print(f"2. Precision: {precision:.2%}")
print(f"3. Recall: {recall:.2%}")
print(f"4. F1-score: {f1:.2%}")


| Metric    |     Mean |    Std Dev |
|:----------|---------:|-----------:|
| Accuracy  | 0.842867 | 0.0040257  |
| Precision | 0.842389 | 0.00410034 |
| Recall    | 0.842867 | 0.0040257  |
| F1-score  | 0.842331 | 0.00403243 |

=== Результаты на Тесте ===
1. Accuracy: 83.14%
2. Precision: 83.10%
3. Recall: 83.14%
4. F1-score: 83.11%


Результаты показывают что собственная имплементация модели в среднем работает примерно на том же уровне качества, что и модель из `sklearn`

### Заключение

Внесённые изменения, включая нормализацию данных и увеличение глубины и числа деревьев в алгоритме, улучшают точность модели. Проведённые эксперименты демонстрируют, что как собственная реализация, так и применение моделей из sklearn, дают схожие результаты.

| Модель                    |      MSE  |        MAE |      $R^2$ |
|:--------------------------|----------:|-----------:|-----------:|
| Sklearn (до улучшения)    | 3.09e+08  | 12029.18   |  0.86      |
| Sklearn (после улучшения) | 2.66e+08  | 10644.36   |  0.88      |
| Собственная имплементация (до улучшения)   | 3.08e+08   | 12143.04    |  0.86      |
| Собственная имплементация (после улучшения)| 2.691e+08  | 10625.08    |  0.88      |

| Модель                    |  Accuracy |  Precision |     Recall |    F1-score |
|:--------------------------|----------:|-----------:|-----------:|-----------:|
| Sklearn (до улучшения)    |   81.38%  |   81.41%   |  81.38%    |  81.16%    |
| Sklearn (после улучшения) |   82.87%  |   82.88%   |  82.87%    |  82.71%    |
| Собственная имплементация (до улучшения)   |   81.26%  |   81.28%   |  81.26%    |  81.04%    |
| Собственная имплементация (после улучшения)|   83.14%  |   83.10%   |  83.14%    |  83.11%    |